In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('datos/productos-asus-lenovo.csv')
df.head()

,nombre,tienda,precio,moneda,simbolo,estrellas,reviews,descripcion,especificaciones,categoria,categoria_path,url,imagen,sku,id
0,2021 ASUS TUF Dash F15,ASUS,NaN,NaN,NaN,4.5,100.0,Windows 10 Home - ASUS recomienda Windows 11 P...,{},Diseño ultra delgado,"['Laptops', 'For-Gaming', 'TUF-Gaming']",https://www.asus.com/ar/laptops/for-gaming/tuf...,https://dlcdnwebimgs.asus.com/gain/769aaa49-03...,15400,15400
1,2021 ASUS TUF Gaming A15,ASUS,NaN,NaN,NaN,4.5,100.0,Windows 11 Home - ASUS recomienda Windows 11 P...,{},Gaming / E-sports,"['Laptops', 'For-Gaming', 'TUF-Gaming']",https://www.asus.com/ar/laptops/for-gaming/tuf...,https://dlcdnwebimgs.asus.com/gain/ed4c3cdc-29...,15366,15366
2,2021 ASUS TUF Gaming F15,ASUS,NaN,NaN,NaN,4.5,100.0,Windows 10 Home - ASUS recomienda Windows 11 P...,{},Gaming / E-sports,"['Laptops', 'For-Gaming', 'TUF-Gaming']",https://www.asus.com/ar/laptops/for-gaming/tuf...,https://dlcdnwebimgs.asus.com/gain/6c49e355-de...,15626,15626
3,2021 ROG Zephyrus G14 GA401IHR-K2053T,ASUS,NaN,NaN,NaN,0.0,0.0,Windows 10 Home \n NVIDIA® GeForce™ GTX™ 1650...,{},2021 ROG Zephyrus G14,"['Notebooks', 'ROG Zephyrus']",https://rog.asus.com/ar/laptops/rog-zephyrus/2...,https://dlcdnwebimgs.asus.com/gain/74C87373-E2...,90NR07H3-M00880,R_90NR07H3-M00880
4,2021 ROG Zephyrus G14 GA401QC-HZ106T,ASUS,NaN,NaN,NaN,0.0,0.0,Windows 10 Home \n NVIDIA® GeForce RTX™ 3050 L...,{},2021 ROG Zephyrus G14,"['Notebooks', 'ROG Zephyrus']",https://rog.asus.com/ar/laptops/rog-zephyrus/2...,https://dlcdnwebimgs.asus.com/gain/1991CB0A-20...,90NR05T6-M01780,R_90NR05T6-M01780


## Columnas a eliminar.

In [3]:
df = df.drop(['moneda', 'simbolo'], axis=1)
df.head(1)

,nombre,tienda,precio,estrellas,reviews,descripcion,especificaciones,categoria,categoria_path,url,imagen,sku,id
0,2021 ASUS TUF Dash F15,ASUS,NaN,4.5,100.0,Windows 10 Home - ASUS recomienda Windows 11 P...,{},Diseño ultra delgado,"['Laptops', 'For-Gaming', 'TUF-Gaming']",https://www.asus.com/ar/laptops/for-gaming/tuf...,https://dlcdnwebimgs.asus.com/gain/769aaa49-03...,15400,15400


## Creación de `score`

In [4]:
def calcular_score_imdb(df, col_estrellas, col_reviews, percentil_minimo=0.75):
    """
    Calcula un score ponderado estilo IMDb para cada fila del DataFrame.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con las columnas de calificaciones y cantidad de reviews.
    col_estrellas : str
        Nombre de la columna que contiene el rating promedio (por ejemplo, 'estrellas').
    col_reviews : str
        Nombre de la columna que contiene la cantidad de reviews.
    percentil_minimo : float, opcional (default=0.75)
        Percentil usado para definir el valor mínimo de reviews (m). 
        Cuanto mayor sea, más exigente será el modelo.

    Retorna:
    --------
    pd.Series
        Columna con el score ponderado estilo IMDb.
    """
    
    # Promedio global de ratings (C)
    C = df[col_estrellas].mean()
    
    # Valor mínimo de reviews para "confiar" (m)
    m = df[col_reviews].quantile(percentil_minimo)
    
    # Evita división por cero
    df = df.copy()
    df[col_reviews] = df[col_reviews].fillna(0)
    df[col_estrellas] = df[col_estrellas].fillna(C)
    
    # Fórmula IMDb
    score = (
        (df[col_reviews] / (df[col_reviews] + m)) * df[col_estrellas] +
        (m / (df[col_reviews] + m)) * C
    )
    
    return score


In [8]:
df['score'] = calcular_score_imdb(df, 'estrellas', 'reviews')
df.sample(5)

,nombre,tienda,precio,estrellas,reviews,descripcion,especificaciones,categoria,categoria_path,url,imagen,sku,id,score
129,Dual GeForce RTX 3060 V2 OC Edition,ASUS,NaN,4.5,100.0,ASUS Dual GeForce RTX™ 3060 V2 OC Edition de 1...,{},NVIDIA,"['Motherboards-Components', 'Graphics-Cards', ...",https://www.asus.com/ar/motherboards-component...,https://dlcdnwebimgs.asus.com/gain/9bf69dcc-53...,16918,16918,3.814621
516,"Legion Pro 5i Gen 10 (16"" Intel) GeForce RTX™ ...",Lenovo,3609668.0,4.8,73.0,"<ul style=""list-style-type: disc;""><li><p>Proc...",{'Procesador': 'Procesador Intel® Core™ Ultra ...,Notebooks,"['ROOTPMI', 'laptops', 'legion-laptops', 'legi...",/p/notebooks/legion-laptops/legion-serie-pro/L...,//p2-ofp.static.pub//fes/cms/2025/10/13/ogyuq0...,83F3CTO1WWAR1_f31ab1f1b6026-4f65-9ed9-0142293b...,NaN,3.834244
312,ROG STRIX LC III 360 ARGB LCD,ASUS,NaN,0.0,0.0,Refrigerador líquido para CPU todo-en-uno ROG ...,{},ROG STRIX LC III 360 ARGB LCD,"['Cooling', 'CPU Liquid Coolers', 'ROG Strix LC']",https://rog.asus.com/ar/cooling/cpu-liquid-coo...,https://dlcdnwebimgs.asus.com/gain/6A08079E-DE...,90RC00W1-M0AAY0,R_90RC00W1-M0AAY0,3.129242
347,ROG Strix GeForce RTX™ 4070 Ti SUPER 16GB GDDR...,ASUS,NaN,0.0,0.0,ROG Strix GeForce RTX™ 4070 Ti SUPER 16GB GDDR...,{},ROG-STRIX-RTX4070TIS-O16G-GAMING,"['Graphics Cards', 'Graphics Cards', 'ROG Strix']",https://rog.asus.com/ar/graphics-cards/graphic...,https://dlcdnwebimgs.asus.com/gain/E0F17D10-DC...,90YV0KG0-M0AA00,R_90YV0KG0-M0AA00,3.129242
367,ROG Strix Scar 17 SE (2022) G733CX-LL052W,ASUS,NaN,0.0,0.0,Windows 11 Home \n NVIDIA® GeForce RTX™ 3080Ti...,{},ROG Strix Scar 17 SE (2022),"['Notebooks', 'ROG Strix']",https://rog.asus.com/ar/laptops/rog-strix/rog-...,https://dlcdnwebimgs.asus.com/gain/575E5FD1-10...,90NR08S3-M002E0,R_90NR08S3-M002E0,3.129242


### Correción de taxto con `html` anidado.

In [16]:
import re
import math
from bs4 import BeautifulSoup

# compilo el regex una sola vez por performance
_RE_SOLO_TXT_NUM = re.compile(r"[^A-Za-zÁÉÍÓÚáéíóúÑñ0-9 ]+")

def limpiar_texto_html(x):
    """
    Limpia texto que puede venir con HTML, \n, símbolos, etc.
    Deja solo letras (incluye acentos), números y espacios.
    
    - Si x es NaN/None -> devuelve None
    - Si x es bytes -> decodifica como utf-8 (con errores ignorados)
    - Si x es lista/dict -> lo convierte a string antes de limpiar
    """
    # 0) NaN/None
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    
    # 1) Bytes -> str
    if isinstance(x, (bytes, bytearray)):
        x = x.decode("utf-8", errors="ignore")
    # 2) Forzar string para tipos raros (listas/dicts, etc.)
    elif not isinstance(x, str):
        x = str(x)

    # 3) Quitar HTML dejando solo texto
    texto = BeautifulSoup(x, "html.parser").get_text(separator=" ")

    # 4) Normalizar espacios (incluye \n, \t)
    texto = re.sub(r"\s+", " ", texto)

    # 5) Dejar solo letras (con acentos), números y espacios
    texto = _RE_SOLO_TXT_NUM.sub("", texto)

    # 6) Strip final y colapsar espacios repetidos
    texto = re.sub(r" {2,}", " ", texto).strip()

    return texto if texto != "" else None


In [19]:
ejemplo = df.descripcion.iloc[479]
ejemplo

'<p><span style="font-size: 20px;"><strong>Estudia de manera más inteligente Juega más duro. Adáptate mejor.</strong></span></p><ul style="list-style-type: disc;"><li><p>Amplia experiencia cinematográfica: tablet 3K de 12,7&quot; y 4 altavoces JBL sintonizados con Dolby Atmos®</p></li><li><p>Rendimiento de última generación: MediaTek Dimensity 8300 con un 60 % más de potencia</p></li><li><p>Google Gemini, Circle to Search con Google</p></li></ul>'

In [20]:
limpiar_texto_html(ejemplo)

'Estudia de manera más inteligente Juega más duro Adáptate mejor Amplia experiencia cinematográfica tablet 3K de 127 y 4 altavoces JBL sintonizados con Dolby Atmos Rendimiento de última generación MediaTek Dimensity 8300 con un 60 más de potencia Google Gemini Circle to Search con Google'

In [21]:
df.descripcion = df.descripcion.fillna('')

In [24]:
df.descripcion = df.descripcion.apply(limpiar_texto_html)